# Dataset migration from JSON to SQLite

In [1]:
## Import dependencies

In [2]:
from utils.tables import *
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker as sm
from sqlalchemy.dialects.postgresql import insert
from tqdm import tqdm
import json

In [4]:
## Create databases and its tables

In [5]:
engine = create_engine('sqlite:///TinyWars.db', echo=True)
Session = sm(bind=engine)

with Session() as session:
    Base.metadata.create_all(engine)
    session.commit()

2025-09-01 11:56:26,976 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-01 11:56:26,977 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2025-09-01 11:56:26,978 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-01 11:56:26,981 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2025-09-01 11:56:26,982 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-01 11:56:26,984 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("cos")
2025-09-01 11:56:26,985 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-01 11:56:26,987 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("cos")
2025-09-01 11:56:26,988 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-01 11:56:26,991 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("maps")
2025-09-01 11:56:26,992 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-01 11:56:26,994 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("maps")
2025-09-01 11:56:26,995 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-01 11:56:

In [6]:
engine = create_engine('sqlite:///TinyWars.db')
Session = sm(bind=engine)
session = Session()

def simple_population(Table, json_path: str) -> None:
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    for id, name in data.items():
        session.merge(Table(id=int(id), name=name))

In [7]:
simple_population(CO, 'original_dataset/cos.json')
simple_population(Map, 'original_dataset/maps.json')
session.commit()

def get_mode(value: int) -> str:
    if value <= 2:
        return GAME_MODES[0]
    if value <= 4:
        return GAME_MODES[1]
    if value <= 10:
        return GAME_MODES[2]
    if value <= 16:
        return GAME_MODES[3]
    if value <= 18:
        return GAME_MODES[4]
    return GAME_MODES[5]

with open('original_dataset/extended_matches.json', 'r') as f:
    matches_data = json.load(f)

with session.no_autoflush:
    users_data = {}
    for match_info in tqdm(matches_data['replayInfoArray'], desc='Populating "users" table'):
        for user_id, resulting_elo in zip(match_info['userIdArray'], match_info['rankScoreArray']):
                users_data[user_id] = resulting_elo
    session.add_all([User(id=key, elo=value) for key, value in users_data.items()])
    session.commit()

        
with session.no_autoflush:
    for match_info in tqdm(matches_data['replayInfoArray'], desc='Populating "matches", "players" and "bans" tables'):

        # Create matches
        match = Match(
            id=match_info['replayId'],
            fow=match_info['warType'] % 2 == 0,
            winner_id=match_info['winnerPlayerIndex'],
            mode=get_mode(match_info['warType']),
            ended=datetime.fromtimestamp(match_info['endTimestamp']),
            map_id=match_info['normalMapId']
        )
        session.add(match)
        
        # Create players
        for player_index, (user_id, co_id, resulting_elo) in enumerate(zip(
            match_info['userIdArray'],
            match_info['pickedCoIdArray'],
            match_info['rankScoreArray']
        ), start=1):
            user = User(id=user_id, elo=resulting_elo)
            session.merge(user)
            player = Player(
                match_id=match_info['replayId'],
                user_id=user_id,
                co_id=co_id,
                id=player_index,
                resulting_elo=resulting_elo
            )
            session.add(player)
        
        # Create bans
        for banned_co_id in set(match_info['bannedCoIdArray']):
            ban = Ban(
                match_id=match_info['replayId'],
                co_id=banned_co_id
            )
            session.add(ban)
    session.commit()

Populating "users" table: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9284/9284 [00:00<00:00, 351389.39it/s]
Populating "matches", "players" and "bans" tables: 9284it [00:09, 956.13it/s] 
